In [1]:
!pip install opencv-python numpy scikit-image matplotlib


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import medial_axis
import cv2 as cv
from time import time
import os

In [3]:
def read_images_collection(path):
    collection = []

    if not os.path.exists(path):
        print("El directorio especificado no existe.")
        return collection

    file_list = os.listdir(path)

    for filename in file_list:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            image_path = os.path.join(path, filename)
            image = cv.imread(image_path)
            if image is not None:
                collection.append(image)
            else:
                print(f"No se pudo leer la imagen: {image_path}")

    return collection

In [4]:
collection_path = "./imgs/cuar"
images_collection = read_images_collection(collection_path)
imgnumber = len(images_collection)

In [7]:
selected_image_index = 1
selected_image = images_collection[selected_image_index]
1

1

In [8]:
homographies = []

# Calcular homografías para cada imagen diferente de la imagen base
for i, image in enumerate(images_collection):
    if i != selected_image_index:
        # Encontrar puntos clave y descriptores para el emparejamiento de características
        orb = cv.ORB_create()
        keypoints1, descriptors1 = orb.detectAndCompute(selected_image, None)
        keypoints2, descriptors2 = orb.detectAndCompute(image, None)
        
        # Realizar el emparejamiento de características
        bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
        matches = bf.match(descriptors1, descriptors2)
        
        # Ordenar los emparejamientos por distancia
        matches = sorted(matches, key=lambda x:x.distance)

        # Obtener los puntos correspondientes en ambas imágenes
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        # Calcular la homografía utilizando RANSAC
        H, _ = cv.findHomography(dst_pts, src_pts, cv.RANSAC)

        # Almacenar la homografía
        homographies.append(H)


1


1

In [10]:
def find_matching_pairs(image1, image2):
    # Inicializar el detector de puntos clave (SIFT, SURF, ORB, etc.)
    orb = cv.ORB_create()

    # Encontrar puntos clave y descriptores para ambas imágenes
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # Realizar el emparejamiento de características
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Filtrar las correspondencias usando RANSAC
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    H, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)

    # Obtener las correspondencias inliers
    inliers = np.where(mask.ravel() == 1)[0]
    inlier_matches = [matches[i] for i in inliers]

    return H, inlier_matches

In [11]:
# Seleccionar dos imágenes consecutivas para encontrar correspondencias
image1 = images_collection[0]
image2 = images_collection[1]

# Encontrar correspondencias y estimar la homografía
H, matches = find_matching_pairs(image1, image2)

# Imprimir la homografía y el número de correspondencias inliers
print("Homografía estimada H_ij:")
print(H)
print("Número de correspondencias inliers:", len(matches))

Homografía estimada H_ij:
[[-2.28024299e-01  2.33632447e-01  3.05936670e+02]
 [-1.24098282e+00  9.60785186e-01  1.76108954e+03]
 [-5.57179049e-04  3.26532858e-04  1.00000000e+00]]
Número de correspondencias inliers: 10


In [6]:
shapes = np.zeros((imgnumber, 3)).astype(int)

for i in range(imgnumber):
    shapes[i, :] = images_collection[i].shape
print(shapes)

[[2992 2992    3]
 [2992 2992    3]
 [2992 2992    3]]
